In [1]:
# импорт библиотек
import pandahouse as ph

# SQL 
**Задача 2.1**  
Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной". <br>Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.
<br>Дана таблица в Clickhouse **default.peas**:

|Название атрибута|Тип атрибута|Смысловое значение|
|:-|:-|:-|
|st_id|int|ID ученика|
|timest|timestamp|Время решения карточки|
|correct|bool|Правильно ли решена горошина?|
|subject|text|Дисциплина, в которой находится горошина|

**Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.**
<br>NB! Под очень усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [2]:
# задаем параметры для подключения к базе данных default на ClickHouse:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database': 'default',
                      'user': 'student', 
                      'password': 'dpo_python_2020'
                      }

In [3]:
# Создали запрос для подсчета количества усердных учеников
q_1 = ''' 
    SELECT COUNT(*) AS amount_of_good_students  --посчитали количество учердных учеников
    FROM  
(--объявили параметр текущего месяца, как последний месяц указанный в таблице, 
--т.к. данных за текущий месяц на момент решения задачи в таблице нет.  
    WITH
    (SELECT MAX(toStartOfMonth(timest)) FROM peas) AS current_month 
--Посчитали количество правильно решенных заданий каждым студентом за текущий месяц 
--и выбрали студентов с 20 и более горошинами 
    SELECT st_id, SUM(correct) AS correct_peas FROM peas
    WHERE toStartOfMonth(timest) = current_month 
    GROUP BY(st_id)
    HAVING correct_peas >= 20
) '''


In [4]:
# отправляем запрос и записываем результат в пандасовский датафрейм
q_test = ph.read_clickhouse(query=q_1, connection=connection_default)
q_test

,amount_of_good_students
0,136


#### Задача 2.2 <a class="anchor" id="Задача-2.2"></a>

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.
<br>Дана таблицы в Clickhouse: **default.peas (см. выше), default.studs:**

|Название атрибута|Тип атрибута|Смысловое значение|
|:-|:-|:-|
|st_id|int|ID ученика|
|test_grp|text|Метка ученика в данном эксперименте|

и **default.final_project_check:**

|Название атрибута|Тип атрибута|Смысловое значение|
|:-|:-|:-|
|st_id|int|ID ученика|
|sale_time|timestamp|Время покупки|
|money|int|Цена, по которой приобрели данный курс|
|subject|text|Дисциплина, на которую приобрели полный доступ|

**Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:**

* ARPU 
* ARPAU 
* CR в покупку 
* СR активного пользователя в покупку 
* CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике
* ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.
<br>Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

In [5]:
q_2= '''
--Расчет метрик
    SELECT test_grp, SUM(revenue)/COUNT(DISTINCT st_id) AS ARPU, SUM(IF(revenue!=0, 1 ,0))/COUNT(st_id) AS CR, 
    SUM(IF(active=1,revenue,0))/SUM (active) AS ARPAU,  
    SUM(If(active=1 AND revenue!=0, active,0))/SUM(active) AS CR_active, 
    SUM(If(math_active=1 AND math_revenue!=0, math_active,0))/SUM(math_active) AS CR_math 
    FROM ( 
--Сформировали общую сводную таблицу, с необходимыми данными для расчета метрик
    SELECT grp.st_id AS st_id, test_grp, active, math_active, revenue, math_revenue FROM studs AS grp 
    LEFT JOIN( 
    SELECT st_id, SUM(correct)>10::int AS active, SUM(IF(subject_learn = 'Math',correct,0))>=2::int AS math_active 
    FROM ( 
--К таблице активности добавили дату последней оплаты и отфильтровали задания выполненые до оплаты
--Задания выпоненные после даты выполнения целевого действия(оплаты) не входят в эксперимент
    SELECT l.st_id AS st_id, timest, last_pay,  correct, l.subject AS subject_learn 
    FROM peas AS l 
    LEFT JOIN  
--Определили дату последней оплаты
    (SELECT st_id, MAX(sale_time) AS last_pay FROM final_project_check GROUP BY st_id )AS r USING(st_id) 
    WHERE timest<=last_pay OR last_pay='1970-01-01 03:00:00' 
    ) AS peas_table 
    GROUP BY st_id 
    ) AS activity ON grp.st_id=activity.st_id 
    LEFT JOIN 
-- Расчитали выручку для каждого оплатившего студента по всем предметам и по математике
    (SELECT st_id, SUM(money) AS revenue, SUM(IF(subject_pay = 'Math',money,0)) AS math_revenue  
    FROM( 
--В таблицу с оплатами добавили дату первой активности и отфильтровали оплату сделанную после первой активности
--Оплата, совершенная до первой активности не входит в эксперимент
    SELECT st_id, sale_time,first_active, money, subject AS subject_pay  
    FROM final_project_check 
    LEFT JOIN  
--Определили дату первой активности для каждого оплатившего студента 
    (SELECT st_id, MIN(timest)  AS first_active 
    FROM peas 
    GROUP BY st_id) As n USING(st_id) 
    WHERE sale_time>first_active 
    ) AS payment  
    GROUP BY st_id) AS revenue ON grp.st_id=revenue.st_id 
    ) AS data_set 
    GROUP BY test_grp '''

In [6]:
q_test_2 = ph.read_clickhouse(query=q_2, connection=connection_default)
q_test_2

,test_grp,ARPU,CR,ARPAU,CR_active,CR_math
0,control,4540.983607,0.049180,5168.067227,0.05042,0.041667
1,pilot,11288.135593,0.105085,25934.065934,0.21978,0.095238
